In [ ]:
# 修改dat file赋予bond和dihedial势能

import os
import sys
import fileinput

di_strenght = [0.2,0.4,0.6,0.8,1.0]
in_dir = 'code/MC_model/python/spinodal_model' # output directory
for ds in di_strenght:
    out_dir = f'spinodal_model_dat_{ds}' # input directory
    
    os.system('mkdir -p %s' % out_dir)
    
    trilinic = True
    bond = '\t1 0.959691 1.88122 3.11032 -0.91359 5.0\n'
    dihedral = f'\t1 {ds} 1 1\n'
    for root, dirs, files in os.walk(in_dir):
        for file in files:
            if file.endswith('.dat')and file[0]!='.':
                in_f = os.path.join(root, file)
                out_f = os.path.join(out_dir, file)
                print(in_f,out_f)
                fin = open(in_f, 'r')
                fout = open(out_f, 'w')
                for id,line in enumerate(fin.readlines(),1):# 25 for bond and 29 for dihedral
                    # print(line)
                    if trilinic:
                        if id == 17:
                            line = line+'0.0 0.0 0.0 xy xz yz\n'
                    if id == 25:
                        line = bond
                    elif id == 29:
                        line = dihedral
                    fout.write(line)
                fin.close()
                fout.close()

In [ ]:
# simulate 

infile_dir = 'infile'
data_dir_ = 'spinodal_model_dat'

di_strenght = [0.2,0.4,0.6,0.8,1.0]


import os

for ds in di_strenght:
    data_dir= data_dir_ +f'_{ds}'
    for iroot,idirs,ifiles in os.walk(infile_dir):
        if len(iroot.split('/'))!=1:
            continue
        for ifile in ifiles:
            if ifile[0] != '.' and 'compress' in ifile:
                in_file = os.path.join(iroot,ifile)
                c = 0
                for droot, ddirs,dfiles in os.walk(data_dir):
                    for dfile in dfiles:
                        if dfile[0] != '.' and dfile.endswith('.dat'):
                            data = os.path.join(droot,dfile)[:-4]
                            data = os.path.abspath(data)
                            o_droot= os.path.abspath(droot)
                            o_droot = o_droot.replace('home','scratch')
                            o_data = os.path.join(o_droot+'/test75',dfile)[:-4]
                            os.system(f'mkdir -p {o_droot+"/test75"}')
                            c+=1
                            cmd = f'mpirun -np 64 tian_jie_lmp -i {in_file} -var file {data} -var ofile {o_data}'
                            print(cmd)
                            # os.system(cmd)
                            break
                            if c>=10:
                                break

# run simulation on cluster

In [5]:
# get simulation batch file 
import os
import numpy as np

job_name = 'Simulation'

infile_dir = 'infile'
data_dir_ = 'spinodal_model_dat'

# di_strenght = [0.2,0.4,0.6,0.8,1.0]
di_strenght = [0.2,0.6,1.0]


bf = open(f'batch_files/{job_name}.sh','w')
bf.write(f'''#!/bin/bash

#SBATCH --partition=batch   # Partition name (batch, highmem_p, or gpu_p)
##SBATCH --gres=gpu:A100:1
#SBATCH --nodes=1
#SBATCH --ntasks=64
#SBATCH --cpus-per-task=1
#SBATCH --mem=128G
#SBATCH --time=2-00:00:00
#SBATCH --job-name={job_name}
#SBATCH --mail-user=jt35560@uga.edu   # Mair eves BOIN, END, FAIL, ALL)
#SBATCH --mail-type=ALL
#SBATCH --output=/home/jt35560/tmp/slurm_log/%j-%x-%N.out

source /home/jt35560/.bashrc
source /home/jt35560/.local/module/lmp.sh
conda activate py311 || exit
ml OpenMPI

cd /home/jt35560/codebase/prj3/batch_files

# Log file to keep track of completed jobs
LOGFILE="{job_name}_log.txt"
# Check if log file exists, if not, create one
if [[ ! -f "$LOGFILE" ]]; then
    touch "$LOGFILE"
fi

# Output log for the jobs
OUTPUT_LOG="{job_name}_output_log.txt"

if [[ ! -f "$OUTPUT_LOG" ]]; then
    touch "$OUTPUT_LOG"
fi

declare -a jobs=(
''')

job_num = 0
import os
comp = 25
for ds in di_strenght:
    data_dir= data_dir_ +f'_{ds}'
    for iroot,idirs,ifiles in os.walk(infile_dir):
        if len(iroot.split('/'))!=1:
            continue
        for ifile in ifiles:
            if ifile[0] != '.' and 'compress' in ifile and not('relax' in ifile):
                in_file = os.path.join(iroot,ifile)
                in_file = os.path.abspath(in_file)
                for droot, ddirs,dfiles in os.walk(data_dir):
                    for dfile in dfiles:
                        if dfile[0] != '.' and dfile.endswith('.dat'):
                            data = os.path.join(droot,dfile)[:-4]
                            data = os.path.abspath(data)
                            o_droot= os.path.abspath(droot)
                            o_droot = o_droot.replace('home','scratch')
                            o_data = os.path.join(o_droot+f'/{job_name}',dfile)[:-4]
                            os.system(f'mkdir -p {o_droot+f"/{job_name}"}')
                            c+=1
                            cmd = f'mpirun -np 64 tian_jie_lmp -i {in_file} -var file {data} -var ofile {o_data} -var comp {comp}'
                            # print(cmd)
                            bf.write("\""+cmd+"\"\n")
                            job_num+=1
                            # os.system(cmd)

                                    
bf.write(')\n')


bf.write('''



# Function to check if a job is already done
is_job_done() {
    job_number=$1
    grep -q "Job $job_number: DONE" "$LOGFILE"
}

# Function to mark a job as done
mark_job_done() {
    job_number=$1
    echo "Job $job_number: DONE" >> "$LOGFILE"
}

# Start executing jobs
for i in "${!jobs[@]}"; do
    job_number=$((i + 1))
    job="${jobs[$i]}"

    if is_job_done $job_number; then
        echo "Job $job_number is already done. Skipping."
        continue
    fi

    # Run the job and log its output
    echo "Starting Job $job_number: $job"
    if eval "$job" >> "$OUTPUT_LOG" 2>&1; then
        echo "Job $job_number completed successfully."
        mark_job_done $job_number
    else
        echo "Job $job_number failed. Check $OUTPUT_LOG for details."
        exit 1
    fi
done

echo "All jobs completed."

''')

bf.close()
print(job_num)

7695


# test load and unload

In [ ]:
# get simulation batch file 
import os
import numpy as np

job_name = 'test_load_unload'

infile_dir = 'infile'
data_dir_ = 'spinodal_model_dat'

di_strength = [0.2,0.4,0.6,0.8,1.0]


bf = open(f'batch_files/{job_name}.sh','w')
bf.write(f'''#!/bin/bash

#SBATCH --partition=batch   # Partition name (batch, highmem_p, or gpu_p)
##SBATCH --gres=gpu:A100:1
#SBATCH --nodes=1
#SBATCH --ntasks=64
#SBATCH --cpus-per-task=1
#SBATCH --mem=128G
#SBATCH --time=2-00:00:00
#SBATCH --job-name={job_name}
#SBATCH --mail-user=jt35560@uga.edu   # Mair eves BOIN, END, FAIL, ALL)
#SBATCH --mail-type=ALL
#SBATCH --output=/home/jt35560/tmp/slurm_log/%j-%x-%N.out

source /home/jt35560/.bashrc
source /home/jt35560/.local/module/lmp.sh
conda activate py311 || exit
ml OpenMPI

cd /home/jt35560/codebase/prj3/batch_files

# Log file to keep track of completed jobs
LOGFILE="{job_name}_log.txt"
# Check if log file exists, if not, create one
if [[ ! -f "$LOGFILE" ]]; then
    touch "$LOGFILE"
fi

# Output log for the jobs
OUTPUT_LOG="{job_name}_output_log.txt"

if [[ ! -f "$OUTPUT_LOG" ]]; then
    touch "$OUTPUT_LOG"
fi

declare -a jobs=(
''')

job_num = 0
import os
for comp in np.arange(10,70,10):
    for ds in di_strength:
        data_dir= data_dir_ +f'_{ds}'
        for iroot,idirs,ifiles in os.walk(infile_dir):
            if len(iroot.split('/'))!=1:
                continue
            for ifile in ifiles:
                if ifile[0] != '.' and 'compress' in ifile and 'relax' in ifile:
                    in_file = os.path.join(iroot,ifile)
                    in_file = os.path.abspath(in_file)
                    c = 0
                    for droot, ddirs,dfiles in os.walk(data_dir):
                        for dfile in dfiles:
                            if dfile[0] != '.' and dfile.endswith('.dat'):
                                data = os.path.join(droot,dfile)[:-4]
                                data = os.path.abspath(data)
                                o_droot= os.path.abspath(droot)
                                o_droot = o_droot.replace('home','scratch')
                                o_data = os.path.join(o_droot+f'/{job_name}',dfile)[:-4]
                                os.system(f'mkdir -p {o_droot+f"/{job_name}"}')
                                c+=1
                                cmd = f'mpirun -np 64 tian_jie_lmp -i {in_file} -var file {data} -var ofile {o_data} -var comp {comp}'
                                print(cmd)
                                bf.write("\""+cmd+"\"\n")
                                job_num+=1
                                # os.system(cmd)
                                if c>=5:
                                    break
                                    
bf.write(')\n')


bf.write('''



# Function to check if a job is already done
is_job_done() {
    job_number=$1
    grep -q "Job $job_number: DONE" "$LOGFILE"
}

# Function to mark a job as done
mark_job_done() {
    job_number=$1
    echo "Job $job_number: DONE" >> "$LOGFILE"
}

# Start executing jobs
for i in "${!jobs[@]}"; do
    job_number=$((i + 1))
    job="${jobs[$i]}"

    if is_job_done $job_number; then
        echo "Job $job_number is already done. Skipping."
        continue
    fi

    # Run the job and log its output
    echo "Starting Job $job_number: $job"
    if eval "$job" >> "$OUTPUT_LOG" 2>&1; then
        echo "Job $job_number completed successfully."
        mark_job_done $job_number
    else
        echo "Job $job_number failed. Check $OUTPUT_LOG for details."
        exit 1
    fi
done

echo "All jobs completed."

''')

bf.close()
print(job_num)

# Post process

In [1]:
# post

import ovito 
import pandas as pd
import numpy as np


def post(file,ofile):
    pipline = ovito.io.import_file(file)
    frame_num = pipline.source.num_frames
    # print(frame_num)
    S = np.zeros((frame_num, 24))

    for i in range(frame_num):
        data = pipline.compute(i)
        cell = np.array(data.cell)
        # print(cell)
        voll = cell[:,:3].diagonal()
        vol = np.prod(voll)
        vxyz = cell[:,3].flatten()
        # print(vxyz)
        # print(vol)
        # calculate the stress tensor
        sxx = data.particles['c_1[1]'].array
        syy = data.particles['c_1[2]'].array
        szz = data.particles['c_1[3]'].array
        sxy = data.particles['c_1[4]'].array
        sxz = data.particles['c_1[5]'].array
        syz = data.particles['c_1[6]'].array
        
        sxxb = data.particles['c_b[1]'].array
        syyb = data.particles['c_b[2]'].array
        szzb = data.particles['c_b[3]'].array
        sxyb = data.particles['c_b[4]'].array
        sxzb = data.particles['c_b[5]'].array
        syzb = data.particles['c_b[6]'].array
        
        sxxd = data.particles['c_d[1]'].array
        syyd = data.particles['c_d[2]'].array
        szzd = data.particles['c_d[3]'].array
        sxyd = data.particles['c_d[4]'].array
        sxzd = data.particles['c_d[5]'].array
        syzd = data.particles['c_d[6]'].array
        
        
        pxx = np.sum(sxx) /(vol * 1e4)
        pyy = np.sum(syy) /(vol * 1e4)
        pzz = np.sum(szz) /(vol * 1e4)
        pxy = np.sum(sxy) /(vol * 1e4)
        pxz = np.sum(sxz) /(vol * 1e4)
        pyz = np.sum(syz) /(vol * 1e4)
        
        pxxb = np.sum(sxxb) /(vol * 1e4)
        pyyb = np.sum(syyb) /(vol * 1e4)
        pzzb = np.sum(szzb) /(vol * 1e4)
        pxyb = np.sum(sxyb) /(vol * 1e4)
        pxzb = np.sum(sxzb) /(vol * 1e4)
        pyzb = np.sum(syzb) /(vol * 1e4)
        
        
        pxxd = np.sum(sxxd) /(vol * 1e4)
        pyyd = np.sum(syyd) /(vol * 1e4)
        pzzd = np.sum(szzd) /(vol * 1e4)
        pxyd = np.sum(sxyd) /(vol * 1e4)
        pxzd = np.sum(sxzd) /(vol * 1e4)
        pyzd = np.sum(syzd) /(vol * 1e4)
        
        
        
        S[i] = [voll[0], voll[1], voll[2], vxyz[0], vxyz[1], vxyz[2], 
                pxx, pyy, pzz, pxy, pxz, pyz, pxxb, pyyb, pzzb, 
                pxyb, pxzb, pyzb, pxxd, pyyd, pzzd, pxyd, pxzd, pyzd]

    pd.DataFrame(S).to_csv(ofile+'.csv',header=['lx','ly','lz','lxy','lxz','lyz','sxx','syy','szz','sxy','sxz','syz','sxxb','syyb','szzb','sxyb','sxzb','syzb','sxxd','syyd','szzd','sxyd','sxzd','syzd'],index=False)


import os

import multiprocessing as mp
from multiprocessing import Manager


num_process =62
mgr = Manager()
jobList = mgr.list([])

data_dir_ = 'spinodal_model_dat'
di_strength = [0.2,0.4,0.6,0.8,1.0]

for ds in di_strength:
    DataDir = data_dir_ +f'_{ds}'
    DumpDataDir = os.path.abspath(DataDir).replace('home','scratch')
    for root,dirs,files in os.walk(DumpDataDir):
        for file in files:
            if file.endswith('dump') and file[0] != '.':
                if not 'center' in file:
                    if 'Simulation' in root: # choose which batch file
                        file = os.path.join(root,file)
                        print(file)
                        ofile = file.replace('prj3','prj3/csv_res')
                        o_dir = os.path.split(ofile)[0]
                        os.system(f'mkdir -p {o_dir}')
                        # post(file,ofile)
                        jobList.append([file,ofile])





def run():
    while True:
        if len(jobList) == 0:
            break;
        job = jobList.pop()
        # print(job)
        post(*job)


# if __name__ =='__main__':

processes = []
for i in range(num_process):
    p = mp.Process(target=run,args=())
    processes.append(p)
    p.start()

for p in processes:
    p.join()
    


/home/jt35560/.conda/envs/py311/lib/python3.11/site-packages/ovito/_extensions/averaging.py:2: UserWarning: Did you accidentally install the OVITO package from the PyPI repository in an Anaconda/Miniconda Python interpreter using the 'pip' command? This will likely lead to conflicts with existing libraries in the Anaconda environment, and import of the OVITO module may fail with an error related to the Qt framework. To fix this, please uninstall the ovito pip package by running 'pip uninstall -y ovito PySide6' and then install the OVITO Anaconda package provided by OVITO GmbH. Visit https://docs.ovito.org/python/introduction/installation.html for further instructions. If you would rather like to ignore this warning message, add the following code to the top of your Python script:

  import warnings
  warnings.filterwarnings('ignore', message='.*OVITO.*PyPI')

  import ovito._extensions.pyscript


/scratch/jt35560/codebase/prj3/spinodal_model_dat_0.2/Simulation/0.30_0.40_0.20_7.y.comp.dump
/scratch/jt35560/codebase/prj3/spinodal_model_dat_0.2/Simulation/0.10_0.00_0.30_7.x.comp.dump
/scratch/jt35560/codebase/prj3/spinodal_model_dat_0.2/Simulation/0.00_0.30_0.50_10.y.comp.dump
/scratch/jt35560/codebase/prj3/spinodal_model_dat_0.2/Simulation/0.30_0.40_0.20_5.z.comp.dump
/scratch/jt35560/codebase/prj3/spinodal_model_dat_0.2/Simulation/0.40_0.60_0.00_10.y.comp.dump
/scratch/jt35560/codebase/prj3/spinodal_model_dat_0.2/Simulation/0.60_0.30_0.10_10.x.comp.dump
/scratch/jt35560/codebase/prj3/spinodal_model_dat_0.2/Simulation/0.10_0.20_0.30_5.z.comp.dump
/scratch/jt35560/codebase/prj3/spinodal_model_dat_0.2/Simulation/0.30_0.60_0.00_7.y.comp.dump
/scratch/jt35560/codebase/prj3/spinodal_model_dat_0.2/Simulation/0.00_0.50_0.40_7.y.comp.dump
/scratch/jt35560/codebase/prj3/spinodal_model_dat_0.2/Simulation/0.20_0.50_0.10_5.z.comp.dump
/scratch/jt35560/codebase/prj3/spinodal_model_dat_0.2/Sim

In [ ]:
import os
import numpy as np
import pandas as pd
from plot_layout import layout
import plotly.graph_objects as go

csv_dir = '/scratch/jt35560/codebase/prj3/csv_res'

# tb = pd.read_csv('csv_res/0.0_0.0_0.0_10.stl.x.dump.csv',sep=',')
# eps = (tb['lx'] - tb['lx'][0])/tb['lx'][0]
# eps = -np.array(eps)
for root, dirs,files in os.walk(csv_dir):
    for file in files:
        if file.endswith('csv') and file[0]!='.':
            res = os.path.join(root, file)
            orient = file.split('.')[-5]
            print(res,orient)
            if len(orient) == 1:
                sorient = orient+orient
            else:
                sorient = orient
            data = pd.read_csv(res,sep=',')
            eps = (data['l'+orient] - data['l'+orient][0])/data['l'+orient][0]
            eps = -np.array(eps)
            data_y = data['s'+sorient]
            data_y = data_y-data_y[0]
            scatter = go.Scatter(x=eps[5:],y=-data_y[5:],mode='markers+lines',name='tol')
            scatterb = go.Scatter(x=eps[5:],y=-data['s'+sorient+'b'][5:],mode='markers+lines',name='bond')
            scatterd = go.Scatter(x=eps[5:],y=-data['s'+sorient+'d'][5:],mode='markers+lines',name='dihedral')
            
            fig = go.Figure([scatter,scatterb,scatterd])
            layout['yaxis']['title']['text'] = '<b>Stress(Gpa)</b>'
            layout['yaxis']['title']['text'] = '<b>Strain</b>'
            layout['showlegend'] = True
            fig.update_layout(layout)
            # fig.show()
            fig.write_image(res[:-4]+'.png')
            # break
            
            